In [29]:
import sys
!{sys.executable} -m pip install lyricsgenius

In [30]:
import lyricsgenius
import json
import pandas as pd
import nltk
from nltk.lm import MLE
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.tokenize import ToktokTokenizer
from nltk import word_tokenize, sent_tokenize 
from nltk.tokenize import RegexpTokenizer
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from string import punctuation
import re

nltk.download('punkt')
from nltk.lm import KneserNeyInterpolated as KNI
from nltk.lm.preprocessing import padded_everygram_pipeline


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nataliya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/nataliya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
sent_tokenize = lambda x: re.split(r'(?<=[^A-Z].[.?]) +(?=[A-Z])', x)
toktok = ToktokTokenizer()
word_tokenize = word_tokenize = toktok.tokenize

**Preparing the data**

In [32]:
genius = lyricsgenius.Genius("Hm2O62KAtRDaiE9AcUkCmQKivJeVn-7-x1Mj7oXDy0wOzvyAn8L4Jx3XmPjlKOmo")
genius.remove_section_headers = True
train_lyrics = ['"Город под подошвой (City Under the Sole)"', "«Где нас нет» (”On the Other Side”)", "До зимы (Before Winter)", "Неваляшка (Tumbler Toy)", "Переплетено (Interlaced)", "В книге всё было по-другому (4 раунд 17ib) (The Book Had It Different)", "Тентакли (Tentacles)", "Восточный Мордор (East Mordor)", "Песенка Гремлина (Gremlin’s Song)", "Признаки Жизни (Signs of Life)", "Детектор лжи (Lie Detector)", "Пролив Дрейка (Drake Passage)", "Башня из слоновой кости (Ivory Tower)", "Биполярочка (Bipolarochka)", "Девочка Пиздец (Fucked Up Girl)", "Последний звонок (Last Call)", "Всего лишь писатель (Just a Writer)", "Привет со дна  (Hello from the Bottom)", "Не от мира сего (Not of This World)", "Больше Бена (Bigga Than Ben)", "Хитиновый покров (Chitin Shell)", "«Полигон» (“Butts”)", "Спонтанное самовозгорание (Spontaneous Combustion)", "Кем ты стал? (What Had You Become?)", "Russky Cockney", "Накануне (On the Eve)", "Darkside", "Жук в муравейнике (Beetle in an anthill)", "Дело нескольких минут (3 раунд 17ib) (A Matter of Minutes)", "Лондон против всех (London Vs. Everyone)"]
text = ""
for index in train_lyrics:
    song = genius.search_song(index, 'Oxxxymiron')
    text += song.lyrics + " "

Searching for ""Город под подошвой (City Under the Sole)"" by Oxxxymiron...
Done.
Searching for "«Где нас нет» (”On the Other Side”)" by Oxxxymiron...
Done.
Searching for "До зимы (Before Winter)" by Oxxxymiron...
Done.
Searching for "Неваляшка (Tumbler Toy)" by Oxxxymiron...
Done.
Searching for "Переплетено (Interlaced)" by Oxxxymiron...
Done.
Searching for "В книге всё было по-другому (4 раунд 17ib) (The Book Had It Different)" by Oxxxymiron...
Done.
Searching for "Тентакли (Tentacles)" by Oxxxymiron...
Done.
Searching for "Восточный Мордор (East Mordor)" by Oxxxymiron...
Done.
Searching for "Песенка Гремлина (Gremlin’s Song)" by Oxxxymiron...
Done.
Searching for "Признаки Жизни (Signs of Life)" by Oxxxymiron...
Done.
Searching for "Детектор лжи (Lie Detector)" by Oxxxymiron...
Done.
Searching for "Пролив Дрейка (Drake Passage)" by Oxxxymiron...
Done.
Searching for "Башня из слоновой кости (Ivory Tower)" by Oxxxymiron...
Done.
Searching for "Биполярочка (Bipolarochka)" by Oxxxymiron.

In [35]:
#splitting into phrases
tr_text = re.split(r'\n', text)

In [36]:
#removing punctuation, putting text into lowercase, deleting empty strings
import string
tr_text = [re.sub(r'[^A-Za-zА-Яа-я0-9ё ]+', '', x) for x in tr_text]

tr_text = [x.lower() for x in tr_text]

tr_text = [x for x in tr_text if x]

In [37]:
#splitting text into words
words = []
for i in range(len(tr_text)):
    t = []
    for j in tr_text[i].split():
        t.append(j)
    words.append(t)

In [38]:
#applying tokenizer to our lyrics split into words
list1 = [' '.join(sent) for sent in words]
tokens = [list(map(str.lower, word_tokenize(sent))) for sent in list1]

In [39]:
#checking tokenizing
tokens[100]

['раз', 'на', 'раз', 'баш', 'на', 'баш', 'чё', 'зассал', 'не', 'пацан']

**2-gram model**

In [40]:
train_data, padded_sents = padded_everygram_pipeline(2, tokens)

In [41]:
from nltk.tokenize.treebank import TreebankWordDetokenizer
detokenize = TreebankWordDetokenizer().detokenize

#defining a function
def generate_sent(model, num_words, random_seed=42):
    content = []
    for token in model.generate(num_words, random_seed=random_seed):
        if token == '<s>':
            continue
        if token == '</s>':
            break
        content.append(token)
    return detokenize(content)

In [42]:
#fitting the model
model = MLE(2)
model.fit(train_data, padded_sents)

In [43]:
#checking the length of our vocabulary
len(model.vocab)

5797

In [44]:
for i in range(20):
    if generate_sent(model,20,random_seed=i) == '':
        continue
    else:
        print(generate_sent(model,20,random_seed=i))

ста икон
ты с каждым вторым перезнакомься
что я был vip до сих пор mc руки к тебе в метро глуховского
my whole life on the best is me
me
на соседей идут следом
рунами да правосудие щедрое хер как мощи в кресло вицемэра метит вам признаки жизни в погреб отпираю ржавый медный замок
вброд или пинап
hacksaw ragh
здесь достигли смены парадигмы
майкчеку кровью харкал
за рупор это круговорот природы червяков доест орёл
и против всех
ак завтра перевернёт всё здесь видишь glenfiddich
школяр в пух перемолот
всё переплетено и мне мученье доставили но пятый годкак я за два пути
как ты хочешь теперь у таких прицел на бал фитиль был всем как игра в чём не яд
подземка не говорят стать толерантным надо жить если за совет братан
но ты не надо детский сад дружный класс дважды два метра но времени в каземате прозябает только будущее и не


**3-gram model**

In [45]:
train_data, padded_sents = padded_everygram_pipeline(3, tokens)

In [46]:
#fitting the model
model = MLE(3)
model.fit(train_data, padded_sents)

In [47]:
for i in range(20):
    if generate_sent(model,20,random_seed=i) == '':
        continue
    else:
        print(generate_sent(model,20,random_seed=i))

своим удивительно просто
чел ты вспомни мы с ней белиберда
был внутри ты забытый на мели
как выстрел в лицо
коров отвык
помереть еретиком
плюс и дева это не побег
эх день простоять мог ли я себе нередко врал
впереди водоворот
и мастурбировал
видел диапозитив
всё предельно серьёзно
избавляя избирателя от самотерзаний
что
dre
до смерти
и как всегда в моих куплетах ктото умрёт
мчится вспять цирк и детсад русрэпа моя crew спецназ йе


**4-gram model**

In [48]:
train_data, padded_sents = padded_everygram_pipeline(4, tokens)

In [49]:
#fitting the model
model = MLE(4)
model.fit(train_data, padded_sents)

In [50]:
for i in range(20):
    if generate_sent(model,20,random_seed=i) == '':
        continue
    else:
        print(generate_sent(model,20,random_seed=i))

пулитцер брать
ходу рановато выводы сами из неё сделали пацаны
но твою ебу под феном и ибупрофеном
впереди водоворот
захотел по красоте рубить стафф по асфальту мимо цемента
оспы
знаю девы приведут к проблемам
life
горестную лыбу бытия ухмылку авгура
ты мой дауншифтинг дня я твоя миссия века
vgb
а ты хоть полумёртвый ты помнишь на том берегу золотое руно
четкий посыл
лучше с редькиным за руку на буерак
в себе щерит волчий оскал рра
кольцо проносит и бросит в жерло


So, generated texts are not really good and I highly doubt that we could call them rap :(

Sources:
1. https://www.kaggle.com/alvations/n-gram-language-model-with-nltk
2. https://www.nltk.org/book/ch01.html
3. http://www.machinelearning.ru/wiki/images/archive/e/e9/20180912152902%21Mmta18-langmodels.pdf
4. https://kite.com/python/docs/nltk.lm
5. https://www.geeksforgeeks.org/tokenize-text-using-nltk-python/